In [10]:
!pip install transformers
!pip install gradio

In [11]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
# Import transformers pipeline
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr
import torch

In [12]:
# Create the summarize function passing in the desired parameter for the most likely sms_text.
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data.
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    features = sms_text_df['text_message']
    # Set the target variable to the "label" column.
    target = sms_text_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

    # Build a pipeline to transform the test set to compare to the training set.
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),  # Transform text to TF-IDF features
        ('svc', LinearSVC())                  # Support Vector Classifier
    ])

    # Fit the model
    pipeline.fit(X_train, y_train)

    return pipeline

In [13]:
# Load the dataset into a DataFrame
sms_text_df = pd.read_csv('SMSSpamCollection.csv')
# Display some of the DataFrame to check if it's loaded properly
sms_text_df.head()


,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [14]:
# Check for null values
# print(sms_text_df.isnull().sum())
# Check for duplicates
# print(sms_text_df.duplicated().sum())
sms_text_df.value_counts()

label  text_message                                                                                                                                                                                                                         
ham    Sorry, I'll call later                                                                                                                                                                                                                   30
       I cant pick the phone right now. Pls send a message                                                                                                                                                                                      12
       Ok...                                                                                                                                                                                                                                    10
       Wen ur lovable bcums angry wid u, dnt take it seriously.. Coz being angry is d most childish n true way of showing deep affection, care n luv!.. kettoda manda... Have nice day da.                                                       4
       Ok.                                                                                                                                                                                                                                       4
                                                                                                                                                                                                                                                ..
       I was about to do it when i texted. I finished a long time ago and showered and er'ything!                                                                                                                                                1
       I wanted to wish you a Happy New Year and I wanted to talk to you about some legal advice to do with when Gary and I split but in person. I'll make a trip to Ptbo for that. I hope everything is good with you babe and I love ya :)     1
       I wanted to ask ü to wait 4 me to finish lect. Cos my lect finishes in an hour anyway.                                                                                                                                                    1
       I want to tell you how bad I feel that basically the only times I text you lately are when I need drugs                                                                                                                                   1
spam   we tried to contact you re your response to our offer of a new nokia fone and camcorder hit reply or call 08000930705 for delivery                                                                                                        1
Name: count, Length: 5169, dtype: int64

In [15]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_text_df)
text_clf

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svc', LinearSVC())])

In [16]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam".
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    prediction = text_clf.predict([text])
    # Create a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    if prediction[0] == "ham":
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'


In [17]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.
# Povide labels for each textbox.
sms_app = gr.Interface(fn=sms_prediction,
                       inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
                       outputs='text')

# Launch the app.
sms_app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca4a9f19d9b63b93c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Test the following text messages.

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.